<img src="https://dt99qig9iutro.cloudfront.net/production/images/header-logo-green.png" alt="QuantInsti Logo">

<h1 style="text-align:center;"> PRM-02 Quantitative Portfolio Management </h1>

<h3 style="text-align:center;"> Author & Presenter: Jay Parmar </h3>

<h5 style="text-align:center;"> Last Updated on: 03/12/2022 </h5>

# Lecture Agenda
- [0. Portfolio Building Blocks](#)
- [1. Invest in a Single Stock](#1.-Invest-in-a-Single-Stock)
    - [a. Returns Calculations](#a.-Returns-Calculations)
        - [i. Single Period Return](#i.-Single-Period-Return)
        - [ii. Holding Period Returns {HPR} / Cumulative Returns](#ii.-Holding-Period-Returns)
        - [iii. Arithmetic Mean Return](#iii.-Arithmetic-Mean-Return)
        - [iv. Annual Return {CAGR}](#iv.-Annualized-Return-{CAGR})
    - [b. Risk Calculations](#b.-Risk-Calculations)
        - [i. Variance](#i.-Variance)
        - [ii. Standard Deviation](#ii.-Standard-Deviation)
    - [c. Invest in Another Stock](#c.-Invest-in-Another-Stock)
- [2. Portfolio Creation {Modern Portfolio Theory Concepts}](#2.-Portfolio-Creation-{Modern-Portfolio-Theory-Concepts})
    - [a. Expected Portfolio Returns Calculation](#a.-Expected-Portfolio-Returns-Calculation)
    - [b. Expected Portfolio Volatility Calculation](#b.-Expected-Portfolio-Volatility-Calculation)
        - [i. Using the first principles](#i.-Calculating-Expected-Portfolio-Volatility-using-the-First-Principles)
        - [ii. Using matrices](#ii.-Calculating-Expected-Portfolio-Volatility-using-Matrices)
    - [c. Correlation Calculation](#c.-Correlation-Calculation)
    - [d. Creation of a Portfolio of Uncorrelated Securities](#d.-Creation-of-a-Portfolio-of-Uncorrelated-Securities)
    - [e. Risk Diversification](#e.-Risk-Diversification)
    - [f. Types of Risks](f.-Types-of-Risks)
        - [i. Unsystematic Risk](#i.-Unsystematic-Risk-/-Diversifiable-Risk)
        - [ii. Systematic Risk](#ii.-Systematic-Risk)
    - [g. Portfolio Optimization using Monte Carlo Simulations](#g.-Portfolio-Optimization-using-Monte-Carlo-Simulations)
        - [i. Minimum Risk Portfolios](#i.-Minimum-Risk-Portfolios)
        - [ii. Maximum Sharpe Portfolios](#ii.-Maximum-Sharpe-Portfolio)
    - [h. Optimal Portfolio Backtest](#h.-Optimal-Portfolio-Backtest)
- [3. Profitability Analysis](#3.-Profitability-Analysis)
    - [a. Sharpe Ratio](#a.-Sharpe-Ratio)
    - [b. Sortino Ratio](#b.-Sortino-Ratio)
    - [c. Treynor Ratio](#c.-Treynor-Ratio)
    - [d. Calmar Ratio](#d.-Calmar-Ratio)
    - [e. Information Ratio](#e.-Information-Ratio)
    - [f. Demo of PyFolio Package](#f.-Demo-of-PyFolio-Library)
- [4. Resources](#4.-Resources)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import yfinance as yf
import plotly.express as px
import plotly.graph_objs as go

%matplotlib inline

warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.options.display.precision = 4
plt.style.use('seaborn-ticks')

In [ ]:
import pyfolio as pf
import cufflinks

cufflinks.go_offline()

In [ ]:
# !pip install pyfolio==0.9.2

## 1. Invest in a Single Stock

In [ ]:
# Define stock 1 name
stock_1 = 'TCS' # IT Companies

In [ ]:
# Read stock 1 data
stock_1_data = pd.read_csv(stock_1 + '.csv', index_col=0, parse_dates=True)

In [ ]:
# Verify the data
stock_1_data

## a. Returns Calculations

### i. Single Period Return

It tells you how much you would get by holding an investment in a single period.

$$ \text{Single Period Return} = R_1 = \frac{\text{Sell price} - \text{Buy price}}{\text{Buy price}} $$

In [ ]:
# Extract prices
sell_price = stock_1_data['Adj Close'].iloc[-1]
buy_price = stock_1_data['Adj Close'].iloc[-2]

# Compute Single Period Return
single_period_returns = (sell_price - buy_price) / buy_price

# Print it
print('Single Period Return is', round(single_period_returns, 4))

#### # Single Period Return for the whole data

In [ ]:
# Calculate Single Period Return
stock_1_data['returns'] = stock_1_data['Adj Close'].pct_change()

# Drop nan values
stock_1_data.dropna(inplace=True)

# Print data
stock_1_data.tail()

### ii. Holding Period Returns / Cumulative Returns

It answers you ***how much you would have earned from the investment over a holding period?***

$$ HPR = (1 + R_1) \times (1 + R_2) \times (1 + R_3) \times \dots \times (1 + R_n) $$

In [ ]:
# Calculate HPR returns
stock_1_data['hpr'] = (1 + stock_1_data['returns']).cumprod()

stock_1_data.head()

In [ ]:
stock_1_data.tail()

In [ ]:
# Extract the gross HPR and subtract 1 to get net HPR
net_hpr = stock_1_data['hpr'].iloc[-1] - 1

print('Net Holding Period Returns is', round(net_hpr, 4))

## Generate Returns Statistics

### iii. Arithmetic Mean Return

The arithmetic average return answers the question: ***What was your return in an average year over a particular period?*** In other words, it tells you what you earned in a typical year.

$$ Arithmetic\ Mean\ Return = \frac{R_1 + R_2 + R_3 + \dots + R_n}{n} $$

We would be using mean return as the **expected return**.

### iv. Annualized Return {CAGR}

An annualized total return is the geometric average amount of money earned by an investment each year over a given time period. The annualized return formula is calculated as a geometric average to show what an investor would earn over a period of time if the annual return was compounded.

It is also known as the Compounded Annual Growth Rate (CAGR).

$$ Annualized\ Returns = (1 + Cumulative\ Return)^\frac{252}{N} - 1 $$

Where:

$ 252 = $ Number of trading days in year

$ N = $ Number of trading days for a strategy

In [ ]:
def generate_returns_statistics(returns_data, stock_name):
    
    #-------------------------------------------------------
    # The following code calculates cumulative returns (HPR)
    #-------------------------------------------------------
    
    # Calculate net cumulative returns
    net_cumulative_returns = (1 + returns_data).cumprod() - 1
    
    # Plot cumulative returns
    net_cumulative_returns.iplot(title=stock_name + ' Cumulative Returns', xTitle='Dates', yTitle='Returns (in %)')

    # Print net returns
    print('The cumulative returns for %s are %.3f%%' % (stock_name, net_cumulative_returns[-1] * 100))
    
    print('*' * 20)
    
    #-------------------------------------------------------
    # The following code calculates mean returns
    #-------------------------------------------------------
    
    # Calculate daily mean returns
    daily_mean_returns = returns_data.mean()
    
    # Print daily mean returns
    print('Daily (arithmetic) mean returns of %s is %.4f%%' % (stock_name, daily_mean_returns * 100))
    
    # Calculate annual mean returns
    annual_mean_returns = daily_mean_returns * 252

    # Print annual mean returns
    print('Annual mean returns of %s is %.4f%%' % (stock_name, annual_mean_returns * 100))
    
    print('*' * 20)
    
    #-------------------------------------------------------
    # The following code calculates annualized returns
    #-------------------------------------------------------
    
    # Define number of trading days
    trading_days = 252
    
    # Calculate total number of data points
    n = len(returns_data)
    
    # Calculate annualized returns (CAGR)
    annualized_returns = ((1 + net_cumulative_returns[-1]) ** (trading_days/n)) - 1
    
    print('The annualized returns for %s are %.3f%%' % (stock_name, annualized_returns * 100))
    
    return (net_cumulative_returns, daily_mean_returns, annual_mean_returns, annualized_returns)

In [ ]:
stock_1_cumulative_returns, stock_1_daily_mean_returns, stock_1_annual_mean_returns, \
stock_1_annualized_returns = generate_returns_statistics(stock_1_data['returns'], stock_1)

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

#### # Questions to ask:-

1. Should we invest in only single stock? If yes, why? If no, why not?
2. Is investing in a single stock risky?

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

## b. Risk Calculations

### i. Variance

Variance measures how far data points are spread out from their mean. Variance is always non-negative. A small variance means the data points are close to their mean, while a large variance signifies highly dispersed dataset. In trading, variance is an important measure, as traders and investors evaluate divergence of returns from mean returns.

$$ Variance = S^2 = \frac{\sum{(x_i - \bar{x})^2}}{n - 1} $$

$$ \text{Annualized Variance} = S^2 \times 252 $$

**Note:** As variance is a squared term, it is unit-less.

### ii. Standard Deviation

The volatility is the standard deviation of the returns of the portfolio. Annualized Volatility can be calculated by multiplying the daily volatility with the square root of number of trading days in a year.

$$ Volatility = Standard\ Deviation = S = \sqrt{Variance} = \sqrt{\frac{\sum{(x_i - \bar{x})^2}}{n - 1}} $$

$$ \text{Annualized Volatility} = S \times \sqrt{252} $$

In [ ]:
def generate_risk_statistics(returns_data, stock_name):
    
    #-------------------------------------------------------
    # The following code calculates variance
    #-------------------------------------------------------
    
    # Calculate daily variance
    daily_variance = returns_data.var()
    
    # Print daily variance
    print('The daily variance of %s is %.4f' % (stock_name, daily_variance))
    
    # Calculate annual variance
    annual_variance = daily_variance * 252
    
    # Print annual variance
    print('The annual variance of %s is %.4f' % (stock_name, annual_variance))
    
    print('*' * 20)
    
    #-------------------------------------------------------
    # The following code calculates standard deviation
    #-------------------------------------------------------
    
    # Calculate daily standard deviation
    daily_std_dev = returns_data.std()
    
    # Print daily standard deviation
    print('The daily std dev (volatility) of %s is %.4f%%' % (stock_name, daily_std_dev * 100))
    
    # Calculate annual standard deviation
    annual_std_dev = daily_std_dev * np.sqrt(252)
    
    # Print annual standard deviation
    print('Annual std dev (volatility) of %s is %.4f%%' % (stock_name, annual_std_dev * 100))
    
    return (daily_variance, annual_variance, daily_std_dev, annual_std_dev)

In [ ]:
stock_1_daily_variance, stock_1_annual_variance, stock_1_daily_std_dev, \
stock_1_annual_std_dev = generate_risk_statistics(stock_1_data['returns'], stock_1)

In [ ]:
# Comparison
comparison = pd.DataFrame()

comparison.loc[stock_1, 'annual_mean_returns'] = round(stock_1_annual_mean_returns * 100, 3)
comparison.loc[stock_1, 'annual_volatility'] = round(stock_1_annual_std_dev * 100, 3)

comparison.head()

#### # Observations:

- Investing in a single asset is risky.

#### # Next Questions:

- Can we do anything to obtain similar returns with less volatility(risk)?

#### # Possible Solution:

- Invest in some other stock.

## c. Invest in Another Stock

In [ ]:
# Define stock 2 name
stock_2 = 'MARUTI' # Automobile giant 

In [ ]:
# Read stock 2 data
stock_2_data = pd.read_csv(stock_2+'.csv', index_col=0, parse_dates=True)

# Compute daily percentage returns for stock 2
stock_2_data['returns'] = stock_2_data['Adj Close'].pct_change()

# Drop nan values
stock_2_data.dropna(inplace=True)

# Verify data
stock_2_data.head()

In [ ]:
# Compute cumulative, mean and annualized returns
stock_2_cumulative_returns, stock_2_daily_mean_returns, stock_2_annual_mean_returns, \
stock_2_annualized_returns = generate_returns_statistics(stock_2_data['returns'], stock_2)

In [ ]:
# Compute risk measures for stock 2
stock_2_daily_variance, stock_2_annual_variance, stock_2_daily_std_dev, \
stock_2_annual_std_dev = generate_risk_statistics(stock_2_data['returns'], stock_2)

#### # Store Values for Comparison

In [ ]:
# Add to the comparison dataframe
comparison.loc[stock_2, 'annual_mean_returns'] = round(stock_2_annual_mean_returns * 100, 3)
comparison.loc[stock_2, 'annual_volatility'] = round(stock_2_annual_std_dev * 100, 3)

comparison.head()

#### # Observations:

- As mean returns increased, volatility also increased.

#### #  Questions to ask:

- Can we do anything to aim similar returns as stock 2, but with a lower risk?

#### # Possible solution:

- Combine both stocks and create a portfolio of them.

---

# 2. Portfolio Creation {Modern Portfolio Theory Concepts}

## a. Expected Portfolio Returns Calculation

$$ Portfolio\ Returns = (w_A * R_A) + (w_B * R_B)$$

Where:

$ w_A = $ Weight in stock A

$ R_A = $ Returns of stock A

$ w_B = $ Weight in stock B

$ R_B = $ Returns of stock B

$$ Portfolio\ Returns = (w_A * R_A) + (w_B * R_B) + (w_C * R_C) + \cdots + (w_N * R_N)$$

In [ ]:
# Define weights for each stock
weights = [0.5, 0.5]

# Define list for stocks mean return
mean_returns = [stock_1_annual_mean_returns, stock_2_annual_mean_returns]

# Compute portfolio returns
expected_portfolio_returns_annual = np.dot(weights, mean_returns)

# Print annual expected returns of a portfolio
print('Expected annual returns of a portfolio with %s and %s is %.3f%%' % 
      (stock_1, stock_2, expected_portfolio_returns_annual * 100))

#### # Questions to ask:

- How would you compute returns of a portfolio with more than two stocks?

## b. Expected Portfolio Volatility Calculation

Unlike portfolio returns, we cannot simply take the weighted average of individual stock variances as shown below:

$$ \text{Portfolio Variance} = w^2_A \times \sigma^2(R_A) + w^2_B \times \sigma^2(R_B)  $$

We would also have to consider the relation between assets while calculating a portfolio variance. Hence, the generalized formula turns out to be:

$$ \text{Portfolio Variance} = w^2_A \times \sigma^2(R_A) + w^2_B \times \sigma^2(R_B) + 2 \times w_A \times w_B \times cov(R_A, R_B) $$

To get the dispersion in portfolio returns in measurable terms, we take the square root of the portfolio variance to get portfolio standard deviation (volatility):

$$ \text{Portfolio Std Dev} = \sqrt{w^2_A \times \sigma^2(R_A) + w^2_B \times \sigma^2(R_B) + 2 \times w_A \times w_B \times cov(R_A, R_B)} $$

Where:-

$ w_A = $ Weight in stock A

$ \sigma^2(R_A) = $ Variance of returns of stock A

$ w_B = $ Weight in stock B

$ \sigma^2(R_B) = $ Variance of returns of stock B

$ cov(R_A, R_B) = $ Corvariance between stock A returns and stock B returns

To calculate the expected volatility of a portfolio with TCS and MARUTI, we use the following formula:

$$ \text{Portfolio Std Dev} = \sqrt{w^2_{stock\_1} \times \sigma^2(R_{stock\_1}) + w^2_{stock\_2} \times \sigma^2(R_{stock\_2}) + 2 \times w_{stock\_1} \times w_{stock\_2} \times cov(R_{stock\_1}, R_{stock\_2})} $$

Substituting:
- TCS with stock_1
- MARUTI with stock_2

$$ \text{Portfolio Std Dev} = \sqrt{w^2_{TCS} \times \sigma^2(R_{TCS}) + w^2_{MARUTI} \times \sigma^2(R_{MARUTI}) + 2 \times w_{TCS} \times w_{MARUTI} \times cov(R_{TCS}, R_{MARUTI})} $$

### i. Calculating Expected Portfolio Volatility using the First Principles

In [ ]:
# Calculate annual variances of both stocks
stock_1_annual_variance = stock_1_data['returns'].var() * 252
stock_2_annual_variance = stock_2_data['returns'].var() * 252

# Print annual variances
print('Anuual variance of %s is %.4f' % (stock_1, stock_1_annual_variance))
print('Anuual variance of %s is %.4f' % (stock_2, stock_2_annual_variance))

#### # Calculate Covariance between Stocks

$$ Covariance = Cov(x, y) = \frac{\sum{(x_i - \bar{x})\ (y_i - \bar{y})}}{n - 1} $$

In [ ]:
# Calculate covariances between stock 1 and stock 2
stock_1_2_covariance = np.cov(stock_1_data['returns'], stock_2_data['returns'])

# Store variance-covariance matrix in pandas dataframe
daily_var_cov_matrix = pd.DataFrame(stock_1_2_covariance, columns=['TCS', 'Maruti'], index=['TCS', 'Maruti'])

# Calculate annual variance-covariance matrix between stock 1 and stock 2
annual_var_cov_matrix = daily_var_cov_matrix * 252

In [ ]:
# Supress scientific notations in Pandas
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Print variance-covariance matrix
annual_var_cov_matrix

In [ ]:
# Extract covariance
tcs_maruti_covarirance = annual_var_cov_matrix.iloc[0, 1]

print('The covariance between %s and %s is %.6f' % (stock_1, stock_2, tcs_maruti_covarirance))

In [ ]:
# Define weights
weight_in_stock_1 = weight_in_stock_2 = 0.5

# Calculate annual portfolio variance
expected_tcs_maruti_variance_annual = ((weight_in_stock_1 ** 2) * stock_1_annual_variance) + \
                                      ((weight_in_stock_2 ** 2) * stock_2_annual_variance) + \
                                      (2 * weight_in_stock_1 * weight_in_stock_2 * tcs_maruti_covarirance)
        
# Calculate annual portfolio volatility
expected_tcs_maruti_vol_annual = np.sqrt(expected_tcs_maruti_variance_annual)

print('Expected annual volatility of portfolio with %s and %s is %.4f%%' % 
      (stock_1, stock_2, expected_tcs_maruti_vol_annual * 100))

### ii. Calculating Expected Portfolio Volatility using Matrices

In [ ]:
# Define weight vector
weights = np.array([0.5, 0.5])

In [ ]:
# Calculate annual portfolio variance
expected_portfolio_variance_annual = weights.T.dot(annual_var_cov_matrix).dot(weights)

# Calculate annual portfolio volatility
expected_portfolio_vol_annual = np.sqrt(expected_portfolio_variance_annual)

print('Expected annual portfolio volatility with %s and %s is %.4f%%' % 
      (stock_1, stock_2, expected_portfolio_vol_annual * 100))

#### # Store Values for Comparison

In [ ]:
# Update the portfolio data in 'comparison' dataframe
comparison.loc[stock_1 + '_' + stock_2, 'annual_mean_returns'] = round(expected_portfolio_returns_annual * 100, 3)
comparison.loc[stock_1 + '_' + stock_2, 'annual_volatility'] = round(expected_portfolio_vol_annual * 100, 3)

comparison.head()

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

#### # Questions to ask:

1. Did investing in two stocks reduced the risk?
2. Can we further reduce the risk?

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

## c. Correlation Calculation

$$ Correlation = r = \frac{Cov(x, y)}{S_x, S_y} $$

In [ ]:
# Correlation between stock 1 and 2
corr = np.corrcoef(stock_1_data['returns'], stock_2_data['returns'])[0, 1]

print('A correlation between %s and %s is %.3f' % (stock_1, stock_2, corr))

## d. Creation of a Portfolio of Uncorrelated Securities

In [ ]:
# Define stock 3
stock_3 = 'MARICO' # FMCG Companies

In [ ]:
# Read stock 3 data
stock_3_data = pd.read_csv(stock_3+'.csv', index_col=0, parse_dates=True)

# Compute daily percentage returns for stock 3
stock_3_data['returns'] = stock_3_data['Adj Close'].pct_change()

# Drop nan values
stock_3_data.dropna(inplace=True)

# Verify the data
stock_3_data.head()

### i. Compute Cumulative Returns

In [ ]:
# Compute cumulative, mean and annualized returns for stock 3
stock_3_cumulative_returns, stock_3_daily_mean_returns, stock_3_annual_mean_returns, \
stock_3_annualized_returns = generate_returns_statistics(stock_3_data['returns'], stock_3)

In [ ]:
# Compute risk measures for stock 3
stock_3_daily_variance, stock_3_annual_variance, stock_3_daily_std_dev, \
stock_3_annual_std_dev = generate_risk_statistics(stock_3_data['returns'], stock_3)

### ii. Correlation Calculation

In [ ]:
# Check correlation between stock 1 and stock 3
corr = np.corrcoef(stock_1_data['returns'], stock_3_data['returns'])[0, 1]

print('A correlation between %s and %s is %.3f' % (stock_1, stock_3, corr))

### iii. Calculate Expected Portfolio Returns

In [ ]:
# Define weights for each stock
weights = [0.5, 0.5]

# Define list for stocks mean return
mean_returns = [stock_1_annual_mean_returns, stock_3_annual_mean_returns]

# Compute portfolio returns
expected_portfolio_returns_annual = np.dot(weights, mean_returns)

# Print annual expected returns of a portfolio
print('Expected annual returns of a portfolio with %s and %s is %.3f%%' % 
      (stock_1, stock_3, expected_portfolio_returns_annual * 100))

### iv. Calculate Expected Portfolio Volatility

In [ ]:
#-------------------------------------------------------
# The following code computes covariance matrix
#-------------------------------------------------------

# Calculate annual variance-covariance matrix between stock 1 and stock 3
annual_var_cov_matrix = np.cov(stock_1_data['returns'], stock_3_data['returns']) * 252

#-------------------------------------------------------
# The following code computes portfolio volatility
#-------------------------------------------------------

# Define weight vector
weights = np.array([0.5, 0.5])

# Calculate annual portfolio variance
expected_portfolio_variance_annual = weights.T.dot(annual_var_cov_matrix).dot(weights)

# Calculate annual portfolio volatility
expected_portfolio_vol_annual = np.sqrt(expected_portfolio_variance_annual)

print('Expected annual portfolio volatility with %s and %s is %.4f%%' % 
      (stock_1, stock_3, expected_portfolio_vol_annual * 100))

In [ ]:
# Update the portfolio data in 'comparison' dataframe
comparison.loc[stock_1 + '_' + stock_3, 'annual_mean_returns'] = round(expected_portfolio_returns_annual * 100, 3)
comparison.loc[stock_1 + '_' + stock_3, 'annual_volatility'] = round(expected_portfolio_vol_annual * 100, 3)

comparison.head()

In [ ]:
# Visualize the data
x = np.arange(4)
fig = plt.figure(figsize=(10, 6))
ax = fig.add_axes([0,0,1,1])
ax.bar(x + 0.00, comparison.annual_mean_returns, color='teal', width = 0.25, label='Annual Expected Returns')
ax.bar(x + 0.25, comparison.annual_volatility, color='g', width = 0.25, label='Annual Expected Volatility')

ax.set_title('Comparison of Expected Returns and Volatility')
ax.set_xticks(x)
ax.set_xticklabels(comparison.index)
ax.legend()

fig.tight_layout()
plt.show()

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

#### # Observations:

- Diversification helped in reducing the portfolio volatility.

#### # Questions to ask:

- Have we achieved our goal?
- How many stocks can we really keep adding to the portfolio?

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

## e. Risk Diversification

In [ ]:
# Read a list of all stocks
all_stocks = pd.read_csv('ind_nifty50list.csv')['Symbol'].to_list()

In [ ]:
all_stocks

In [ ]:
# Create empty dataframe to store all stock data
all_data = pd.DataFrame()

# Download data for all stocks
for stock in all_stocks:
    
    # Download data for each stock
    stock_data = yf.download(stock+'.NS', start='2015-1-1', end='2020-8-28')
    
    # Append stock data to all_data
    all_data[stock] = stock_data['Adj Close']
else:
    print('Data fetched for all stocks!')

In [ ]:
# Print data
all_data.head()

In [ ]:
# Compute daily returns for all stocks
daily_returns = all_data.pct_change()

# Print daily returns
daily_returns.head()

In [ ]:
# Dictionary to hold expected standard deviation of portfolios
sd = {}

# Define number of stocks in each portfolio
number_of_stocks = np.arange(1, len(daily_returns.columns)+1)

# Iterate through each portfolio
for num_stocks in number_of_stocks:
    
    # Create equal weights for each constituent in the stock
    weights = np.full(num_stocks, 1/num_stocks)
    
    # Extract data from whole dataset
    stock_data = daily_returns.iloc[:, :num_stocks]
    
    # Create covariance matrix
    cov_mat = stock_data.cov()
    
    # Calculate annual covariance matrix
    cov_mat = cov_mat * 252
    
    # Calculate expected portfolio volatility
    expected_portfolio_variance = weights.T.dot(cov_mat).dot(weights)
    expected_portfolio_std_dev = np.sqrt(expected_portfolio_variance)
    
    # Store in dictionary
    sd[num_stocks] = round(expected_portfolio_std_dev * 100, 3)
else:
    print('Computations Completed!')

std_dev_of_different_portfolios = pd.Series(sd)

In [ ]:
# Plot Std Dev vs Number of Stocks
std_dev_of_different_portfolios.iplot(xTitle='Number of stocks in a Portfolio', 
                                      yTitle='Expected Portfolio Volatility (%)')

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

<p>
<p>

#### # Observations

- As the number of stocks increases in a portfolio, the risk decreases
- After certain number of stocks in the portfolio, adding more stocks does not reduce the risk
- No matter how many stocks we add in the portfolio, the risk cannot be eliminated totally

## f. Types of Risks

$$ Total\ Risk = Unsystematic\ Risk + Systematic\ Risk $$ 

### i. Unsystematic Risk / Diversifiable Risk 

In [ ]:
daily_returns.corr()

In [ ]:
fig = px.imshow(daily_returns.corr(), width=850, height=850, color_continuous_scale='RdBu_r')
fig.show()

### ii. Systematic Risk

Also known as market risk, a systematic risk is a part of the total risk that is caused by factors beyond the control of a specific company. It cannot be diversified away by holding a large number of securities. It is caused by factors that are external to organizations.



## g. Portfolio Optimization using Monte Carlo Simulations

In [ ]:
new_stocks = ['TITAN', 'ITC', 'TCS', 'BPCL', 'DRREDDY', 'BAJFINANCE']

portfolio_stocks_returns = daily_returns[new_stocks]

In [ ]:
portfolio_stocks_returns

In [ ]:
fig = px.imshow(portfolio_stocks_returns.corr(), width=850, height=850, color_continuous_scale='RdBu_r')
fig.show()

In [ ]:
portfolio_stocks_returns.dropna(inplace=True)

portfolio_stocks_returns.head()

In [ ]:
# Calculate annual mean returns
annual_mean_returns = portfolio_stocks_returns.mean() * 252

# Calculate annual variance-covariance matrix
annual_cov_matrix = portfolio_stocks_returns.cov() * 252

In [ ]:
np.random.seed(11)

weights = np.random.dirichlet(np.ones(len(new_stocks)), size=3500)

In [ ]:
weights[0:3]

In [ ]:
port_data = pd.DataFrame(columns=new_stocks + ['expected_returns', 'expected_volatility'])

# Compute (annual) expected returns for all portfolios
all_portfolio_expected_returns = np.dot(weights, annual_mean_returns)

# Compute (annual) expected risk for all portfolios
all_portfolio_expected_risk = [np.sqrt(x.T.dot(annual_cov_matrix).dot(x)) for x in weights]

In [ ]:
# Store each weight vector in a dataframe
for i in range(0, len(new_stocks)):
    port_data[new_stocks[i]] = weights[:, i] * 100

In [ ]:
# Store risk and return profile for each portfolio in the dataframe
port_data['expected_returns'] = all_portfolio_expected_returns * 100
port_data['expected_volatility'] = np.array(all_portfolio_expected_risk) * 100

In [ ]:
port_data

In [ ]:
fig = px.scatter(port_data, x="expected_volatility", y="expected_returns", color='expected_returns', 
                 render_mode="svg")
fig.show()

### i. Minimum Risk Portfolios

In [ ]:
# Minimum Risk Portfolio
min_risk_portfolios = port_data.loc[port_data.expected_volatility == port_data.expected_volatility.min()]

# Print minimum volatility portfolio
min_risk_portfolios

### ii. Maximum Sharpe Portfolio

In [ ]:
# Create a new column with maximum returns per unit of risk
port_data['max_returns/risk'] = port_data['expected_returns'] / port_data['expected_volatility']

# Find the portfolio with maximum returns per unit of risk
new_sharpe_portfolios = port_data.loc[port_data['max_returns/risk'] == port_data['max_returns/risk'].max()]

# Print max Sharpe portfolios
new_sharpe_portfolios

In [ ]:
# Extract X and Y coordinates for plotting - Minimum Risk Portfolio
min_x = min_risk_portfolios.expected_volatility.iloc[0]
min_y = min_risk_portfolios.expected_returns.iloc[0]

# Extract X and Y coordinates for plotting - Maximum Sharpe Portfolio
sharpe_x = new_sharpe_portfolios.expected_volatility.iloc[0]
sharpe_y = new_sharpe_portfolios.expected_returns.iloc[0]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=port_data['expected_volatility'],
                         y=port_data['expected_returns'],
                         mode="markers+text",
                         marker=dict(color=port_data['expected_returns']),
                         name='Portfolio',
                         showlegend=False))

fig.add_trace(go.Scatter(x=[min_x],
                         y=[min_y],
                         mode="markers",
                         marker=dict(color='SkyBlue', size=10),
                         marker_symbol='star',
                         name='Minimum Volatility Portfolio',
                         showlegend=False))

fig.add_trace(go.Scatter(x=[sharpe_x],
                         y=[sharpe_y],
                         mode="markers",
                         marker=dict(color='Green', size=10),
                         marker_symbol='cross',
                         name='Maximum Sharpe Portfolio',
                         showlegend=False))

fig.update_layout(title='Various Portfolios', xaxis_title='Expected Volatility', yaxis_title='Expected Returns')

fig.show()

## h. Optimal Portfolio Backtest

In [ ]:
# Create empty dataframe
backtest_df = pd.DataFrame()

# Download new data for all constituents and store in a dataframe
for stock in new_stocks:
    
    data = yf.download(stock+'.NS', start='2021-1-1', end='2022-04-24')
    
    data['returns'] = data['Adj Close'].pct_change()
    
    data.dropna(inplace=True)
    
    backtest_df[stock] = data['returns']
else:
    print('Computations Completed!')

In [ ]:
backtest_df

In [ ]:
# Define equal allocation
equal_weights = 1 / len(new_stocks)

# Calculate daily portfolio retursn - equal allocation
backtest_df['equal_allocation_returns'] = (backtest_df * equal_weights).sum(axis=1)

backtest_df.head()

In [ ]:
# Calculate cumulative returns for equal allocation
cumulative_returns_equal_allocation = (1 + backtest_df['equal_allocation_returns']).cumprod() - 1

print('Cumulative returns of a portfolio with equal allocations is %.3f%%' % (cumulative_returns_equal_allocation[-1] * 100))

# Plot cumulative returns
cumulative_returns_equal_allocation.iplot(xTitle='Dates', yTitle='Returns (%)', title='Portfolio with Equal Allocation')

In [ ]:
# Define optimal weights - They come from optimization we did above
optimized_weights = list(new_sharpe_portfolios[new_stocks].values[-1] / 100)

# Calculate daily portfolio retursn - equal allocation
backtest_df['optimized_returns'] = (backtest_df[new_stocks] * optimized_weights).sum(axis=1)

backtest_df.head()

In [ ]:
# Calculate cumulative returns for optimal weights
cumulative_returns_optimized = (1 + backtest_df['optimized_returns']).cumprod() - 1

print('Cumulative returns of a portfolio with optimal allocations is %.3f%%' % (cumulative_returns_optimized[-1] * 100))

# Plot cumulative returns
cumulative_returns_optimized.iplot(xTitle='Dates', yTitle='Returns (%)', title='Portfolio with Optimized Allocation')

---

# 3. Profitability Analysis

## a. Sharpe Ratio

The Sharpe ratio is the excess return calculated as total returns less the risk-free rate of return per unit of volatility. Generally, risk-free return is the return on the risk-free assets such as government bonds. The excess returns are due to the 'extra risk' taken by the investor on investing in risky assets.

It tells whether the returns on a portfolio are due to good investment decision or the result of excessive risk taken. Higher Sharpe ratio is always preferable over the lower ones.

The Sharpe Ratio can be used to compare the portfolio with the benchmark to get to know how your portfolio is repaying for the risk taken on the investment.

$$ Sharpe\ Ratio\ =\ \frac{R_p - R_f}{\sigma_p} $$

Where:

$ R_p $ = Portfolio Returns

$ R_f $ = Risk-free Returns

$ \sigma_p $ = Standard deviation of the portfolio returns

In [ ]:
# Define annualized risk free rate of return
risk_free_rate = 0.00

In [ ]:
# Calculate numerator
excess_returns = stock_1_data['returns'] - risk_free_rate

# Calculate denominator
std_portfolio_returns = stock_1_data['returns'].std()

# Calculate Sharpe Ratio
daily_sharpe = excess_returns.mean() / std_portfolio_returns

# Calculate Annualized Sharpe Ratio
ann_sharpe = daily_sharpe * np.sqrt(252)

print("The Sharpe ratio is %.2f" % ann_sharpe)

## b. Sortino Ratio
In the Sortino ratio, the denominator of the Sharpe ratio, the total standard deviation is replaced with the downside deviation. The downside deviation is the standard deviation of negative asset return.

It differentiates the harmful volatility from the total volatility by using the standard deviation of negative returns only. Since an investor is concerned only about the downside volatility, Sortino ratio is a good measure in comparing the highly volatile portfolios whereas the Sharpe ratio is better at analyzing portfolios with low volatility. The probability of large loss will be low if the value of the Sortino ratio is high.

$$ Sortino\ Ratio\ =\ \frac{R_p - R_f}{\sigma_d} $$

Where:

$ R_p $ = Portfolio Returns
<br>$ R_f $ = Risk-free Returns
<br>$ \sigma_d $ = Standard deviation of the negative asset returns

In [ ]:
# Calculate numerator
excess_returns = stock_1_data['returns'] - risk_free_rate

# Calculate denominator
std_negative_returns = excess_returns[excess_returns < 0].std()

# Compute sortino ratio
sortino_ratio = (excess_returns.mean() / std_negative_returns) * np.sqrt(252)

print('The sortino ratios is %.2f' % (sortino_ratio))

## c. Treynor Ratio
Treynor Ratio is the variation in the denominator of the Sharpe ratio by replacing the total standard deviation with the beta of the portfolio. It also highlights the risk-adjusted performance of the portfolio. Higher the Treynor ratio, more suitable the investment is. The ratio is based on historical returns data, it is not necessary it will replicate in the future. The higher ratio tells that investment is good but it does not quantify how much good the investment is.

$$ Treynor\ Ratio\ =\ \frac{R_p - R_f}{\beta_p} $$

Where:

$ R_p $ = Portfolio Returns
<br>$ R_f $ = Risk-free Returns
<br>$ \beta_p $ = Portfolio's Beta

In [ ]:
# Read benchmark data to calculate beta
benchmark_data = pd.read_csv('Benchmark_data.csv', index_col=0, parse_dates=True)

# Calculate daily benchmark returns
benchmark_data['returns'] = benchmark_data['Close'].pct_change()

# Drop nan values
benchmark_data.dropna(inplace=True)

# Calculate beta
covariance_value = np.cov(stock_1_data['returns'], benchmark_data['returns'])[0, 1] * 252
market_variance = benchmark_data['returns'].var() * 252

beta = covariance_value / market_variance

print('The beta is %.2f' % (beta))

In [ ]:
daily_treynor = (stock_1_data['returns'].mean() - (risk_free_rate/252)) / beta

annual_treynor = daily_treynor * np.sqrt(252)

print('The Treynor ratio is %.2f' % annual_treynor)

## d. Calmar Ratio
It measures the performance of an investment fund compared to its risk. It's a function of the portfolio's annualized rate of return versus its maximum drawdown. It is commonly calculated using the past 36 months data and calculated as follows:

$$ Calmar\ Ratio = \frac{Annualized(R_p)}{MaxDD_p} $$

Where:

$ Annualized(R_p) $ = Annualized Returns
<br>$ MaxDD_p $ = Maximum Drawdown

In [ ]:
# Calculate Maximum Drawdown
# Cumulative product of portfolio returns
cumprod_ret = (stock_1_data['returns'] + 1).cumprod() * 100

# Convert the index in datetime format
cumprod_ret.index = pd.to_datetime(cumprod_ret.index)

# Define a variable trough_index to store the index of lowest value before new high
trough_index = (np.maximum.accumulate(cumprod_ret) - cumprod_ret).idxmax()

# Define a variable peak_index to store the index of maximum value before largest drop
peak_index = cumprod_ret.loc[:trough_index].idxmax()

# Calculate the maximum drawdown using the given formula
maximum_drawdown = 100 * (cumprod_ret[trough_index] - cumprod_ret[peak_index]) / cumprod_ret[peak_index]

print('The maximum drawdown is %.2f%%' % (maximum_drawdown))

# Calculate Calmar Ratio
calmar_ratio = (stock_1_annualized_returns * 100) / abs(maximum_drawdown)

print('Calmar ratio is %.2f' % (calmar_ratio))

## e. Information Ratio
Information ratio tells the portfolio's return in excess of the benchmark's return with respect to the volatility of these returns. It tells an investor how much excess return is generated from the excess risk taken with respect to its benchmark. A higher ratio implies that the fund is more consistent and better performing. The range between 0.4-0.6 is considered good and the value greater than 1 is considered excellent but is found quite rare.

$$ \text{Information Ratio} = \frac{R_p - R_b}{\sigma(R_p - R_b)} $$

Where:

$ R_p $ = Portfolio Returns
<br>$ R_b $ = Benchmark Returns
<br>$ \sigma(R_p - R_b) $ = Tracking error / Standard Deviation of excess returns

In [ ]:
# Calculate numerator
excess_returns = stock_1_data['returns'] - benchmark_data['returns']

# Calculate denominator
std_excess_returns = excess_returns.std()

# Compute information ratio
information_ratio = (excess_returns.mean() / std_excess_returns) * np.sqrt(252)

print('The information ratio is %.2f' % (information_ratio))

## f. Demo of PyFolio Library

In [ ]:
pf.create_simple_tear_sheet(stock_1_data['returns'], benchmark_rets=benchmark_data['returns'])

---

# 4. Resources

- [Portfolio Management Of Multiple Strategies Using Python](https://blog.quantinsti.com/portfolio-management-strategy-python/)
- [Portfolio Optimization Methods](https://blog.quantinsti.com/portfolio-optimization-methods/)
- [Portfolio Analysis: Performance Measurement And Evaluation](https://blog.quantinsti.com/portfolio-analysis-performance-measurement-evaluation/)
- [Calculating The Covariance Matrix And Portfolio Variance](https://blog.quantinsti.com/calculating-covariance-matrix-portfolio-variance/)
- [Optimal Portfolio Construction Using Machine Learning](https://blog.quantinsti.com/optimal-portfolio-construction-machine-learning/)
- [Sharpe Ratio: Calculation, Application, Limitations](https://blog.quantinsti.com/sharpe-ratio-applications-algorithmic-trading/)
- [Volatility And Measures Of Risk-Adjusted Return With Python](https://blog.quantinsti.com/volatility-and-measures-of-risk-adjusted-return-based-on-volatility/)
---

<h3 style="text-align:center;"> Thank You </h3>

<br>